# Downloading the dataset from Kaggle using API

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

Dataset URL: https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset
License(s): copyright-authors
 99% 2.68G/2.70G [00:30<00:00, 106MB/s] 
100% 2.70G/2.70G [00:30<00:00, 95.7MB/s]


# Extracting the data from the zipfile

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/new-plant-diseases-dataset.zip')
zip_ref.extractall('/content')
zip_ref.close()

# Importing the necessary libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout

# Data Preprocessing

Creating generators for the training and testing datasets


In [ ]:
train_ds=keras.utils.image_dataset_from_directory(
    directory='/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train',
    labels ='inferred',
    label_mode ='categorical',
    class_names=None,
    color_mode="rgb",
    shuffle=True,
    seed=None,
    validation_split=None,
    batch_size=32,
    subset=None,
    interpolation="bilinear",
    crop_to_aspect_ratio=False,
    follow_links=False,
    image_size=(128,128)
)

validate_ds=keras.utils.image_dataset_from_directory(
    directory='/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid',
    labels ='inferred',
    label_mode ='categorical',
    class_names=None,
    color_mode="rgb",
    shuffle=True,
    seed=None,
    validation_split=None,
    batch_size=32,
    subset=None,
    interpolation="bilinear",
    crop_to_aspect_ratio=False,
    follow_links=False,
    image_size=(128,128)
)

Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.


# Buiding the CNN Model

In [ ]:
model=Sequential()

#Adding the convolutional layers
model.add(Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=(128,128,3)))
model.add(Conv2D(filters=32,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

model.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu',input_shape=(128,128,3)))
model.add(Conv2D(filters=64,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

model.add(Conv2D(filters=128,kernel_size=3,padding='same',activation='relu',input_shape=(128,128,3)))
model.add(Conv2D(filters=128,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

model.add(Conv2D(filters=256,kernel_size=3,padding='same',activation='relu',input_shape=(128,128,3)))
model.add(Conv2D(filters=256,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

model.add(Conv2D(filters=512,kernel_size=3,padding='same',activation='relu',input_shape=(128,128,3)))
model.add(Conv2D(filters=512,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Dropout(0.2))

In [ ]:
#Flattening the model
model.add(Flatten())

In [ ]:
#Adding Dense layer
model.add(Dense(units=1024,activation='relu'))

In [ ]:
#Output layer
model.add(Dense(units=38,activation='softmax'))

Compiling the Model


In [ ]:
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 63, 63, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 61, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                   

Training the Model

In [ ]:
ds=model.fit(train_ds,validation_data=validate_ds,epochs=10)

Epoch 1/10
1811/2197 [=======================>......] - ETA: 23:28 - loss: 1.2002 - accuracy: 0.6432

Evaluating the model

In [ ]:
tr_loss,tr_acc=model.evaluate(train_ds)

In [ ]:
print(tr_loss,tr_acc)

In [ ]:
val_loss, val_acc=model.evaluate(validate_ds)

In [ ]:
print(val_loss,val_acc)

#Visualizing

In [ ]:
e=[i for i in range(1,11)]
plt.plot(e,ds.history['accuracy'],label='Training Accuracy')
plt.plot(e,ds.history['val_accuracy'],color='red',label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Visualization of Accuracy")
plt.show()

Other metrics of Evaluation

In [ ]:
class_name=validate_ds.class_names
print(class_name)

In [ ]:
test_ds=keras.utils.image_dataset_from_directory(
    directory='/content/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/valid',
    labels ='inferred',
    label_mode ='categorical',
    class_names=None,
    color_mode="rgb",
    shuffle=False,
    seed=None,
    validation_split=None,
    batch_size=32,
    subset=None,
    interpolation="bilinear",
    crop_to_aspect_ratio=False,
    follow_links=False,
    image_size=(128,128)
)

# Prediction

In [ ]:
y_pred=model.predict(test_ds)

In [ ]:
predicted_ds=tf.argmax(y_pred,axis=1)

In [ ]:
y_true=tf.argmax(tf.concat([i for x,i in test_ds],axis=0),axis=1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(y_true,predicted_ds,target_names=class_name))

Confusion matrix Visualization

In [ ]:
c=confusion_matrix(y_true, predicted_ds)
plt.figure(figsize=(30,30))
sns.heatmap(c)
plt.xlabel("Predicted Class",fontsize=25)
plt.ylabel("Actual Class",fontsize=25)
plt.title("Confusion matrix",fontsize=35)


# Visualizing Image of Test DataSet

In [ ]:
import cv2
image_path='/content/test/test/AppleScab1.JPG'
img=cv2.imread(image_path)
plt.imshow(img)
plt.title("Test Image")
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.title("Test Image")
plt.xticks([])
plt.yticks([])
plt.show()

# Testing the Model

In [ ]:
import numpy as np

In [ ]:
image=tf.keras.preprocessing.image.load_img(image_path,target_size=(128,128))
arr=tf.keras.preprocessing.image.img_to_array(image)
arr=np.array([arr])
print(arr.shape)

In [ ]:
pred=model.predict(arr)


Model Prediction

In [ ]:
res=np.argmax(pred)
print(class_name[res])
